In [ ]:
#import keras
import json
import sys
import tensorflow as tf
from tensorflow.keras.layers import Input
import numpy as np
import argparse
import cv2
#from wide_resnet import WideResNet
from tensorflow.keras.utils import get_file
import face_recognition

In [ ]:
age_model = tf.keras.models.load_model('../data/age_model_16_11_2021.h5')
age_model.summary()

In [ ]:
gender_model = tf.keras.models.load_model('../data/gender_model_16_2021.h5')
gender_model.summary()

In [ ]:
# import libraries
import cv2
import face_recognition

# Get a reference to webcam 
video_capture = cv2.VideoCapture(0)

# Initialize variables
face_locations = []

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all the faces in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)

    # Display the results
    for top, right, bottom, left in face_locations:
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

    img= cv2.resize(rgb_frame, (128, 128))
    img=np.array([img]).reshape((1, 128,128,3))
    #predictions 
    age_prediction = age_model.predict(img)
    gender_prediction = gender_model.predict(img)
    #decoding 
    index = np.argmax(age_prediction)
    decoding = {0:'0-2', 1:'4-6', 2:'8-13',3:'15-20',4:'25-32',5:'38-43',6:'48-53',7:'60+'}
    
    predicted_genders = gender_prediction[0]
    
    gen = "F" if predicted_genders[0] > 0.5 else "M"
    pred = str(decoding[index])+" "+str(gen)
    #print(pred)
    cv2.putText(frame, pred,(face_locations[0][3],face_locations[0][0]) , cv2.FONT_HERSHEY_SIMPLEX,0.7, (2, 255, 255), 2)

    # Display the resulting image
    cv2.imshow('Video', frame)
    
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()